In [63]:
import json
import os
from pycocotools.coco import COCO
from shutil import copyfile

In [26]:
TREE_DIR = os.path.join(ROOT_DIR, os.getcwd(), 'datasets','trees')

# merge tree labels

In [24]:
with open(os.path.join(TREE_DIR, 'all_data', 'coco_annotations.json')) as json_file:  
    tree_data = json.load(json_file)

In [25]:
tree_data['categories']

[{'supercategory': 'Trees1', 'id': 1, 'name': 'Trees1'},
 {'supercategory': 'Not trees 1', 'id': 2, 'name': 'Not trees 1'},
 {'supercategory': 'Trees2', 'id': 3, 'name': 'Trees2'},
 {'supercategory': 'Not trees 2', 'id': 4, 'name': 'Not trees 2'}]

In [11]:
tree_data['categories'] = [{'supercategory': 'Trees', 'id': 1, 'name': 'Trees'},
                           {'supercategory': 'Not Trees', 'id': 2, 'name': 'Not Trees'}]

In [12]:
for ann in tree_data['annotations']:
    if ann['category_id'] == 3:
        ann['category_id'] = 1
    elif ann['category_id'] == 4:
        ann['category_id'] = 2

In [ ]:
with open(os.path.join(TREE_DIR, 'all_data', 'clean_coco_annotations.json'), 'w') as outfile:
    json.dump(tree_data, outfile)

# download images

In [ ]:
import requests

coco = COCO("{}/{}/clean_coco_annotations.json".format(TREE_DIR, 'all_data'))

save_dir = os.path.join(TREE_DIR, 'all_data')

for img_id, content in coco.imgs.items():
    img_data = requests.get(content['file_name']).content
    with open(os.path.join(save_dir, f'{img_id}.jpg'), 'wb') as handler:
        handler.write(img_data)

# update filename for coco

In [13]:
with open(os.path.join(TREE_DIR, 'all_data', 'clean_coco_annotations.json')) as json_file:  
    clean_tree_data = json.load(json_file)

In [17]:
for img in clean_tree_data['images']:
    img['file_name'] = img['id'] + '.jpg'

In [19]:
with open(os.path.join(TREE_DIR, 'all_data', 'clean_coco_annotations.json'), 'w') as outfile:
    json.dump(clean_tree_data, outfile)

# Split to train and validation

In [31]:
train_pct = .90
val_pct = 1 - train_pct
all_images = [f for f in os.listdir(os.path.join(TREE_DIR, 'all_data')) if f.endswith('jpg')]

In [59]:
train_num = round(len(all_images) * train_pct)
train_imgs = all_images[:train_num]
val_imgs = [img for img in all_images if img not in train_imgs]

In [41]:
with open(os.path.join(TREE_DIR, 'all_data', 'clean_coco_annotations.json')) as json_file:  
    annotation = json.load(json_file)

In [49]:
train_annotation = annotation.copy()
val_annotation = annotation.copy()

In [56]:
train_annotation['images'] = [info for info in train_annotation['images'] if info['file_name'] in train_imgs]
train_annotation['annotations'] = [info for info in train_annotation['annotations'] if info['image_id'] + '.jpg' in train_imgs]

val_annotation['images'] = [info for info in val_annotation['images'] if info['file_name'] in val_imgs]
val_annotation['annotations'] = [info for info in val_annotation['annotations'] if info['image_id'] + '.jpg' in val_imgs]


In [65]:
# write the split files out
train_dir = os.path.join(TREE_DIR,'train')
val_dir = os.path.join(TREE_DIR,'val')

In [67]:
for img in train_imgs:
    src = os.path.join(TREE_DIR, 'all_data', img)
    copyfile(src, os.path.join(train_dir, img))

with open(os.path.join(TREE_DIR, 'train', 'coco_annotations.json'), 'w') as outfile:
    json.dump(train_annotation, outfile)

In [69]:
for img in val_imgs:
    src = os.path.join(TREE_DIR, 'all_data', img)
    copyfile(src, os.path.join(val_dir, img))

with open(os.path.join(TREE_DIR, 'val', 'coco_annotations.json'), 'w') as outfile:
    json.dump(val_annotation, outfile)